In [ ]:
!python -V

In [35]:
import pandas as pd

In [36]:
import pickle

In [37]:
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [50]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")


In [40]:
mlflow.set_experiment("nyc-taxi-experiment")

2022/07/17 19:50:58 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [41]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [42]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [43]:
len(df_train), len(df_val)

(73908, 61921)

In [44]:
df_train['PU_DO'] = df_train['PULocationID'].astype('str') + '_' + df_train['DOLocationID'].astype('str')
df_val['PU_DO'] = df_val['PULocationID'].astype('str') + '_' + df_val['DOLocationID'].astype('str')

In [45]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [46]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [47]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

2022/07/17 19:51:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f17a5b6cd2884b398c19025a10a34c08', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


7.758715206462274

In [48]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [49]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

2022/07/17 19:51:27 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID d4fa0112b22e414bb0c86ca9ef6e33a0. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'copy_X\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'fit_intercept\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'max_iter\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'normalize\', \'old_value\': None, \'new_value\': \'deprecated\'}, {\'key\': \'positive\', \'old_value\': None, \'new_value\': \'False\'}, {\'key\': \'precompute\', \'old_value\': None, \'new_value\': \'False\'}, {\'key\': \'random_state\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'selection\', \'old_value\': None, \'new_value\': \'cyclic\

In [51]:
import xgboost as xgb

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
X_train

<73908x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 147816 stored elements in Compressed Sparse Row format>

In [52]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[15:00:52] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.28486                                                                                                                  
[1]	validation-rmse:7.96123                                                                                                                   
[2]	validation-rmse:6.99758                                                                                                                   
[3]	validation-rmse:6.70492                                                                                                                   
[4]	validation-rmse:6.60354                                                                                                                   
[5]	validation-rmse:6.55399                                                  

[55]	validation-rmse:6.43681                                                                                                                  
[56]	validation-rmse:6.43738                                                                                                                  
[57]	validation-rmse:6.43840                                                                                                                  
[58]	validation-rmse:6.43927                                                                                                                  
[59]	validation-rmse:6.43919                                                                                                                  
[60]	validation-rmse:6.43880                                                                                                                  
[61]	validation-rmse:6.44021                                                                                                                  

[7]	validation-rmse:13.56598                                                                                                                  
[8]	validation-rmse:12.92433                                                                                                                  
[9]	validation-rmse:12.33460                                                                                                                  
[10]	validation-rmse:11.79545                                                                                                                 
[11]	validation-rmse:11.30362                                                                                                                 
[12]	validation-rmse:10.85452                                                                                                                 
[13]	validation-rmse:10.44615                                                                                                                 

[64]	validation-rmse:6.52506                                                                                                                  
[65]	validation-rmse:6.52127                                                                                                                  
[66]	validation-rmse:6.51781                                                                                                                  
[67]	validation-rmse:6.51490                                                                                                                  
[68]	validation-rmse:6.51231                                                                                                                  
[69]	validation-rmse:6.50982                                                                                                                  
[70]	validation-rmse:6.50731                                                                                                                  

[121]	validation-rmse:6.45929                                                                                                                 
[122]	validation-rmse:6.45863                                                                                                                 
[123]	validation-rmse:6.45825                                                                                                                 
[124]	validation-rmse:6.45763                                                                                                                 
[125]	validation-rmse:6.45718                                                                                                                 
[126]	validation-rmse:6.45696                                                                                                                 
[127]	validation-rmse:6.45665                                                                                                                 

...

[28]	validation-rmse:6.43402                                                                                                                  
[29]	validation-rmse:6.43341                                                                                                                  
[30]	validation-rmse:6.43196                                                                                                                  
[31]	validation-rmse:6.43048                                                                                                                  
[32]	validation-rmse:6.42956                                                                                                                  
[33]	validation-rmse:6.42817                                                                                                                  
[34]	validation-rmse:6.42655                                                                                                                  

[85]	validation-rmse:6.39091                                                                                                                  
[86]	validation-rmse:6.39100                                                                                                                  
[87]	validation-rmse:6.39077                                                                                                                  
[88]	validation-rmse:6.39023                                                                                                                  
[89]	validation-rmse:6.39090                                                                                                                  
[90]	validation-rmse:6.39060                                                                                                                  
[91]	validation-rmse:6.39047                                                                                                                  

[142]	validation-rmse:6.39254                                                                                                                 
[143]	validation-rmse:6.39293                                                                                                                 
[144]	validation-rmse:6.39298                                                                                                                 
[145]	validation-rmse:6.39355                                                                                                                 
[146]	validation-rmse:6.39342                                                                                                                 
[147]	validation-rmse:6.39337                                                                                                                 
[148]	validation-rmse:6.39338                                                                                                                 

[25]	validation-rmse:6.54391                                                                                                                  
[26]	validation-rmse:6.54251                                                                                                                  
[27]	validation-rmse:6.54067                                                                                                                  
[28]	validation-rmse:6.53872                                                                                                                  
[29]	validation-rmse:6.53706                                                                                                                  
[30]	validation-rmse:6.53547                                                                                                                  
[31]	validation-rmse:6.53425                                                                                                                  

KeyboardInterrupt: 

In [53]:
mlflow.xgboost.autolog(disable=True)

In [54]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[19:52:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	vali

[270]	validation-rmse:6.38786
[271]	validation-rmse:6.38755
[272]	validation-rmse:6.38727
[273]	validation-rmse:6.38673
[274]	validation-rmse:6.38660
[275]	validation-rmse:6.38649
[276]	validation-rmse:6.38615
[277]	validation-rmse:6.38595
[278]	validation-rmse:6.38585
[279]	validation-rmse:6.38558
[280]	validation-rmse:6.38529
[281]	validation-rmse:6.38510
[282]	validation-rmse:6.38480
[283]	validation-rmse:6.38455
[284]	validation-rmse:6.38429
[285]	validation-rmse:6.38410
[286]	validation-rmse:6.38377
[287]	validation-rmse:6.38341
[288]	validation-rmse:6.38321
[289]	validation-rmse:6.38288
[290]	validation-rmse:6.38241
[291]	validation-rmse:6.38218
[292]	validation-rmse:6.38187
[293]	validation-rmse:6.38155
[294]	validation-rmse:6.38099
[295]	validation-rmse:6.38073
[296]	validation-rmse:6.38059
[297]	validation-rmse:6.38034
[298]	validation-rmse:6.38019
[299]	validation-rmse:6.38001
[300]	validation-rmse:6.37986
[301]	validation-rmse:6.37969
[302]	validation-rmse:6.37924
[303]	vali

[544]	validation-rmse:6.33730
[545]	validation-rmse:6.33707
[546]	validation-rmse:6.33694
[547]	validation-rmse:6.33680
[548]	validation-rmse:6.33659
[549]	validation-rmse:6.33672
[550]	validation-rmse:6.33653
[551]	validation-rmse:6.33657
[552]	validation-rmse:6.33651
[553]	validation-rmse:6.33631
[554]	validation-rmse:6.33623
[555]	validation-rmse:6.33601
[556]	validation-rmse:6.33609
[557]	validation-rmse:6.33599
[558]	validation-rmse:6.33582
[559]	validation-rmse:6.33582
[560]	validation-rmse:6.33561
[561]	validation-rmse:6.33555
[562]	validation-rmse:6.33554
[563]	validation-rmse:6.33540
[564]	validation-rmse:6.33525
[565]	validation-rmse:6.33519
[566]	validation-rmse:6.33517
[567]	validation-rmse:6.33522
[568]	validation-rmse:6.33518
[569]	validation-rmse:6.33510
[570]	validation-rmse:6.33491
[571]	validation-rmse:6.33472
[572]	validation-rmse:6.33458
[573]	validation-rmse:6.33443
[574]	validation-rmse:6.33427
[575]	validation-rmse:6.33401
[576]	validation-rmse:6.33395
[577]	vali

[818]	validation-rmse:6.31366
[819]	validation-rmse:6.31363
[820]	validation-rmse:6.31342
[821]	validation-rmse:6.31328
[822]	validation-rmse:6.31317
[823]	validation-rmse:6.31322
[824]	validation-rmse:6.31317
[825]	validation-rmse:6.31318
[826]	validation-rmse:6.31317
[827]	validation-rmse:6.31317
[828]	validation-rmse:6.31309
[829]	validation-rmse:6.31290
[830]	validation-rmse:6.31278
[831]	validation-rmse:6.31271
[832]	validation-rmse:6.31266
[833]	validation-rmse:6.31270
[834]	validation-rmse:6.31273
[835]	validation-rmse:6.31271
[836]	validation-rmse:6.31268
[837]	validation-rmse:6.31260
[838]	validation-rmse:6.31253
[839]	validation-rmse:6.31239
[840]	validation-rmse:6.31230
[841]	validation-rmse:6.31227
[842]	validation-rmse:6.31215
[843]	validation-rmse:6.31216
[844]	validation-rmse:6.31220
[845]	validation-rmse:6.31214
[846]	validation-rmse:6.31200
[847]	validation-rmse:6.31178
[848]	validation-rmse:6.31167
[849]	validation-rmse:6.31149
[850]	validation-rmse:6.31131
[851]	vali

In [55]:
logged_model = 'runs:/0385f607dc8e40249af1c2da7515da6c/models_mlflow'

loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model


[19:56:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 0385f607dc8e40249af1c2da7515da6c

In [56]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

[19:56:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [57]:
y_pred = xgboost_model.predict(valid)
y_pred

array([14.940359 ,  7.1591063, 14.152481 , ..., 13.56066  ,  6.3787494,
        8.290052 ], dtype=float32)

In [58]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2022/07/17 19:56:49 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
/Users/mohammad/opt/anaconda3/envs/mlflow-exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
